# Data Collection for Model Prediction with Random Forest, XGBoost, and a Neural Network

## 1. Loading the games DataFrame and Selecting the Relevant Columns

In the first block of cells, we load the games DataFrame and create a games_new DataFrame containing only the columns we'll need for the models.

Throughout the notebook, we add new columns to games_new.

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

base_dir = Path.cwd()
games = pd.read_csv(base_dir / "all_games.csv")

In [ ]:
games['game_date'] = pd.to_datetime(games['game_date'])

In [ ]:
games_new = games[['game_id', 'game_date', 'team_name_home', 'team_name_away', 'matchup_home', 'wl_home']].copy()
games_new

## 2. Win-Loss Records for games_new

Since the games DF contains win-loss records of the teams *after* the game, we subtract the appropriate columns by 1 to prevent data leakage. 

In [ ]:
games_new['record_home_wins'] = games['record_home_wins'].copy()
games_new.loc[games_new['wl_home'] == 'W', 'record_home_wins'] = games_new['record_home_wins'] - 1
games_new['record_home_losses'] = games['record_home_losses'].copy()
games_new.loc[games_new['wl_home'] == 'L', 'record_home_losses'] = games_new['record_home_losses'] - 1
games_new['record_away_wins'] = games['record_away_wins'].copy()
games_new.loc[games_new['wl_home'] == 'L', 'record_away_wins'] = games_new['record_away_wins'] - 1
games_new['record_away_losses'] = games['record_away_losses'].copy()
games_new.loc[games_new['wl_home'] == 'W', 'record_away_losses'] = games_new['record_away_losses'] - 1

## 3. Elo Ratings

The next goal is to add margin-adjusted Elo ratings of each team as columns of games_new. For this, we start from the first NBA game ever played in 1947! But we only care about the 2013-14 season and onwards.

Each team starts with a rating of 1200. The home team has a 114 Elo advantage, and teams playing the second night of a back-to-back have their Elo reduced by around 20 (there is a small variance depending on whether the back-to-backs were played at home or away). 

I developed this in a previous project: https://github.com/Matija-Sreckovic/NBA-Simple-Prediction-Models

The formula for the new Elo rating after the game is:

$$E_{new} = E_{old} + P \times G \times (result - e_r),$$ where:

- $P = 15$ for regular season games, and $P = 10$ for playoff games
- $G = (1 + N_{game}/N_{median})^{1/3}$, where $N_{game}$ is the game's margin and $N_{median}$ the median margin of victory this season; or $G = 1$ if it's the first game of the season.
- $result = 1$ if the team won, $0$ if the team lost 
- $e_r = q_{team} / (q_{team} + q_{opponent})$, where $q_{team} = 10^{Elo_{team}/400}, q_{opponent} = 10^{Elo_{opponent}/400}$ (stands for "expected result")

I found this formula at https://www.aussportstipping.com/sports/nba/elo_ratings/. I tried to tune the $1/3$ parameter but this was more or less the best value.

In [ ]:
games_new['elo_home'] = 1200
games_new['elo_away'] = 1200

In [ ]:
teams = games['team_name_home'].unique()
team_elos = {team : 1200.0 for team in teams}
median_margin = 0
margins_this_season = np.zeros(73, dtype=int)

In [ ]:
# True for home, False for away
previous_game_dates = {team : [datetime.datetime(1,1,1), True] for team in teams}

In [ ]:
def change_elo_G_cbrt(team_1_elo, team_2_elo, plus_minus_home, median_margin, is_reg_seas, btb_hh, btb_ha, btb_ah, btb_aa):
    """
    This function takes in the home team's Elo (team_1_elo) and the away team's Elo (team_2_elo) and some other parameters - plus_minus_home is the
    game margin, median_margin is the median margin of victory this season, is_reg_seas is True if it's a reg. seas. game and False if it's the playoffs,
    btb_hh/ha/ah/aa is True/False depending on whether a team is playing a back-to-back and if they're home-home, home-away, away-home, away-away.
    
    The function returns the new Elo ratings.
    
    The parameters 114, 18.5 and 1.1 were tuned in a previous project (NBA-Simple-Prediction-Models). 
    See https://github.com/Matija-Sreckovic/NBA-Simple-Prediction-Models
    """
    team_1_elo_factored = team_1_elo + 114
    team_2_elo_factored = team_2_elo
    if btb_hh:
        team_1_elo_factored -= 18.5
    if btb_ha:
        team_1_elo_factored -= 18.5*1.1
    if btb_ah:
        team_2_elo_factored -= 18.5*1.1
    if btb_aa:
        team_2_elo_factored -= 18.5*1.21
    first_score = 0
    if plus_minus_home > 0:
        first_score = 1
    G = 1
    if median_margin != 0:
        G = pow((1 + (abs(plus_minus_home)/median_margin)),1.0/3)
    q_a = pow(10, team_1_elo_factored/400)
    q_b = pow(10, team_2_elo_factored/400)
    e_a = q_a/(q_a+q_b)
    if is_reg_seas:
        return team_1_elo + 15 * G * (first_score - e_a), team_2_elo + 15 * G * (e_a - first_score)
    else: return team_1_elo + 10 * G * (first_score - e_a), team_2_elo + 10 * G *(e_a - first_score)

In [ ]:
def row_processing_G_cbrt(row, previous_row, margins_this_season, median_margin, team_elos, previous_game_dates):
    """
    We take in a row of the games table, the previous row to check if the current game is the first game of the season, the margins of victory this season
    to compute the median, the team elos at this point in time, and each of the teams' previous game dates, to check if it's a back-to-back game.
    
    We return the row of the games table *with the Elos before the game added*, and the other parameters updated.
    """
    date = row['game_date']
    home_team = row['team_name_home']
    away_team = row['team_name_away']
    home_elo = team_elos[home_team]
    away_elo = team_elos[away_team]
    row['elo_home'] = team_elos[home_team]
    row['elo_away'] = team_elos[away_team]
    btb_home_home = False
    btb_home_away = False
    btb_away_home = False
    btb_away_away = False
    home_previous_date = previous_game_dates[home_team]
    away_previous_date = previous_game_dates[away_team]
    previous_date = previous_row['game_date']
    # If it's a new season, set the margins to 0.
    if (date.month >= 10 and previous_date.month <= 9) or (date.year == 1999 and previous_date.year == 1998):
        margins_this_season = np.zeros(73, dtype=int)
    game_margin = abs(row['plus_minus_home'])
    margins_this_season[int(game_margin) - 1] += 1
    median_margin = np.median(margins_this_season)
    if home_previous_date == [date - datetime.timedelta(days=1), True]:
        btb_home_home = True
    elif home_previous_date == [date - datetime.timedelta(days=1), False]:
        btb_home_away = True
    elif away_previous_date == [date - datetime.timedelta(days=1), True]:
        btb_away_home = True
    elif away_previous_date == [date - datetime.timedelta(days=1), False]:
        btb_away_away = True
    is_reg = True
    if row['season_type'] == 'Playoffs':
        is_reg = False
    team_elos[home_team], team_elos[away_team] = change_elo_G_cbrt(home_elo, away_elo, row['plus_minus_home'], median_margin, is_reg, btb_home_home, btb_home_away, btb_away_home, btb_away_away)
    previous_game_dates[home_team] = [date, True]
    previous_game_dates[away_team] = [date, False]
    return row, margins_this_season, median_margin, team_elos, previous_game_dates

In [ ]:
# Add Elo ratings as columns of the games_new table...
games_new.reset_index(drop=True, inplace=True)
games['plus_minus_home'] = games['pts_home'] - games['pts_away']
games_new['plus_minus_home'] = games['plus_minus_home']
games_new['season_type'] = games['season_type']
games_new
for idx, row in games_new.iterrows():
    games_new.iloc[idx], margins_this_season, median_margin, team_elos, previous_game_dates = row_processing_G_cbrt(row,  games_new.iloc[row.name - 1] if row.name > 0 else row, margins_this_season, median_margin, team_elos, previous_game_dates)
games_new

## 4. Checkpoint and Win-Loss %

We record what we've done so far, define a base directory, and create home_wl% and away_wl% columns in the games_new DF.

In [ ]:
# Write games_new onto a csv file in case I screw something up later
games_new.to_csv(base_dir / "games_new.csv")

In [3]:
#import libraries, set base directory, load games_new
import pandas as pd
import datetime
import numpy as np
import math
from IPython.display import clear_output
import unicodedata
from rapidfuzz import fuzz, process
from pathlib import Path

base_dir = Path.cwd()

games_new = pd.read_csv(base_dir / "games_new.csv")

In [4]:
# Compute win-loss % of each (home) team in each row before the game.
games_new['home_wl%'] = 0.5
games_new.loc[games_new['record_home_wins'] + games_new['record_home_losses'] > 0, 'home_wl%'] = games_new['record_home_wins'].astype(float)/(games_new['record_home_wins'].astype(float) + games_new['record_home_losses'].astype(float))
games_new

,Unnamed: 0,game_id,game_date,team_name_home,team_name_away,matchup_home,wl_home,record_home_wins,record_home_losses,record_away_wins,record_away_losses,elo_home,elo_away,plus_minus_home,season_type,home_wl%
0,0,24600001,1946-11-01,Toronto Huskies,New York Knicks,HUS vs. NYK,L,0,0,0,0,1200.000000,1200.000000,-2,Regular Season,0.500000
1,1,24600003,1946-11-02,St. Louis Bombers,Pittsburgh Ironmen,BOM vs. PIT,W,0,0,0,0,1200.000000,1200.000000,5,Regular Season,0.500000
2,2,24600002,1946-11-02,Providence Steamrollers,Boston Celtics,PRO vs. BOS,W,0,0,0,0,1200.000000,1200.000000,6,Regular Season,0.500000
3,3,24600004,1946-11-02,Chicago Stags,New York Knicks,CHS vs. NYK,W,0,0,1,0,1200.000000,1209.876218,16,Regular Season,0.500000
4,4,24600005,1946-11-02,Detroit Falcons,Washington Capitols,DEF vs. WSB,L,0,0,0,0,1200.000000,1200.000000,-17,Regular Season,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70895,70895,22400384,2024-12-16,Detroit Pistons,Miami Heat,DET vs. MIA,W,10,16,13,10,1025.628280,1301.064204,1,Regular Season,0.384615
70896,70896,22400385,2024-12-16,Brooklyn Nets,Cleveland Cavaliers,BRK vs. CLE,L,10,15,22,4,1132.806850,1343.690267,-29,Regular Season,0.400000
70897,70897,22400386,2024-12-16,Toronto Raptors,Chicago Bulls,TOR vs. CHI,L,7,19,11,15,1027.362566,1188.929098,-1,Regular Season,0.269231
70898,70898,22400387,2024-12-16,Sacramento Kings,Denver Nuggets,SAC vs. DEN,L,13,13,13,10,1267.454051,1380.783789,-1,Regular Season,0.500000


In [5]:
# Do the same for the away teams...
games_new['away_wl%'] = 0.5
games_new.loc[games_new['record_away_wins'] + games_new['record_away_losses'] > 0, 'away_wl%'] = games_new['record_away_wins'].astype(float)/(games_new['record_away_wins'].astype(float) + games_new['record_away_losses'].astype(float))
games_new

,Unnamed: 0,game_id,game_date,team_name_home,team_name_away,matchup_home,wl_home,record_home_wins,record_home_losses,record_away_wins,record_away_losses,elo_home,elo_away,plus_minus_home,season_type,home_wl%,away_wl%
0,0,24600001,1946-11-01,Toronto Huskies,New York Knicks,HUS vs. NYK,L,0,0,0,0,1200.000000,1200.000000,-2,Regular Season,0.500000,0.500000
1,1,24600003,1946-11-02,St. Louis Bombers,Pittsburgh Ironmen,BOM vs. PIT,W,0,0,0,0,1200.000000,1200.000000,5,Regular Season,0.500000,0.500000
2,2,24600002,1946-11-02,Providence Steamrollers,Boston Celtics,PRO vs. BOS,W,0,0,0,0,1200.000000,1200.000000,6,Regular Season,0.500000,0.500000
3,3,24600004,1946-11-02,Chicago Stags,New York Knicks,CHS vs. NYK,W,0,0,1,0,1200.000000,1209.876218,16,Regular Season,0.500000,1.000000
4,4,24600005,1946-11-02,Detroit Falcons,Washington Capitols,DEF vs. WSB,L,0,0,0,0,1200.000000,1200.000000,-17,Regular Season,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70895,70895,22400384,2024-12-16,Detroit Pistons,Miami Heat,DET vs. MIA,W,10,16,13,10,1025.628280,1301.064204,1,Regular Season,0.384615,0.565217
70896,70896,22400385,2024-12-16,Brooklyn Nets,Cleveland Cavaliers,BRK vs. CLE,L,10,15,22,4,1132.806850,1343.690267,-29,Regular Season,0.400000,0.846154
70897,70897,22400386,2024-12-16,Toronto Raptors,Chicago Bulls,TOR vs. CHI,L,7,19,11,15,1027.362566,1188.929098,-1,Regular Season,0.269231,0.423077
70898,70898,22400387,2024-12-16,Sacramento Kings,Denver Nuggets,SAC vs. DEN,L,13,13,13,10,1267.454051,1380.783789,-1,Regular Season,0.500000,0.565217


In [6]:
# Checkpoint! :)
games_new.to_csv(base_dir / "games_new_wl%.csv")

## 5. Adding the OffRtg-DefRtg-Gamescore Ratings to games_new

In this section, we add the columns 'rating_home' and 'rating_away' to games_new, by more or less copying the code from https://github.com/Matija-Sreckovic/NBA-Prediciton-Model.

### GameScore, Off/DefRtg

I'll try to explain how the rating system works here. It uses two catch-all advanced stats of a box score: a player's **GameScore** (**GmSc**) and the difference between his **OffRtg** and **DefRtg**. The formula for **GmSc** is:

$$ GmSc = PTS + 0.4 \times FG - 0.7 \times FGA - 0.4\times(FTA - FT) + 0.7 \times ORB + 0.3 \times DRB + STL + 0.7 \times AST + 0.7 \times BLK - 0.4 \times PF - TOV.$$

On the other hand, **OffRtg** and **DefRtg** for a *team* are the number of points a team scores/allows per 100 possessions. For a *player*, the formula is much more complicated (see https://www.basketball-reference.com/about/ratings.html), but according to its creator, Dean Oliver, the author of ["Basketball on Paper"](https://www.amazon.com/Basketball-Paper-Rules-Performance-Analysis/dp/1574886886):

**OffRtg:** "Individual offensive rating is the number of points produced by a player per hundred total individual possessions. In other words, 'How many points is a player likely to generate when he tries?'"

**DefRtg:** "The core of the Defensive Rating calculation is the concept of the individual Defensive Stop. Stops take into account the instances of a player ending an opposing possession that are tracked in the boxscore (blocks, steals, and defensive rebounds), in addition to an estimate for the number of forced turnovers and forced misses by the player which aren't captured by steals and blocks."

### The Rating System Itself - 1 Game

A **score** is assigned to each player after each game he played in.

We take in each player's GmSc and difference between OffRtg and DefRtg (henceforth "Rtg"). We compare them to all other scores of players in all of the last 5 years - for example, if each is 99th percentile, the player gets a coefficient of 0.99 for GmSc and Rtg.

To get the player's *unweighted rating*, we set $\textup{rating}_{\textup{unweighted}} = 0.2 \times \textup{coeff}_{\textup{GmSc}} + 0.8 \times \textup{coeff}_{\textup{Rtg}}.$ The value $0.2$ was tuned.

To get the player's *weighted rating* we multiply the unweighted rating by a **usage rate coefficient** (we get the player's USG% and assign a coefficient by comparing it to all other games in the last 5 years, similarly to how we obtain the Gmsc and Rtg coefficients) and **minutes_coefficient** (if a player played at least 35 minutes, the coefficient is 1). Precisely, $\textup{coeff}_{\textup{min}} = \min(1, 6.85 \times \frac{\textup{player's minutes played}}{\textup{total team minutes}}).$ In total, $$\textup{rating}_{\textup{weighted}} = \textup{coeff}_{\textup{USG%}} \times \textup{coeff}_{\textup{min}} \times (0.2 \times \textup{coeff}_{\textup{GmSc}} + 0.8 \times \textup{coeff}_{\textup{Rtg}}).$$

### Players' Long-Term Ratings and a Team's Rating before a Game

The score that we compute for a player after each game is added to his rating sum *this season* and his *5-year rating sum*. Both are divided by the number of games played to get the *average rating this season* and *average 5-year rating*. There are also *last season's average ratings* and *average ratings from 2 seasons ago*.

To compute a player's rating before a game, we use a weighted average of the 4 parameters above (this season's avg rating so far, last season's avg rating, avg rating from 2 seasons ago, 5-year avg rating). The weights depend on how many games the player has played this season and the previous 2 seasons. For example, if a player has played 15 games this year, 55 games last year and 0 games 2 years ago, the formula would be:

$$\textup{player game rating} = 0.5 \times \textup{rating this season} + 0.3 \times \textup{rating last season} + 0 \times \textup{rating 2 seasons ago} + 0.2 \times \textup{rating in the last 5 years}.$$

The weights were selected a bit arbitrarily, but tuning did not improve the model's performance.

At the end of this process, we get a vector of ratings for about 10-15 players; we take all **uninjured** players for the game! Call this vector $v_{\textup{ratings}}.$ The team's score is the dot product of this vector with the **vector of players' usage estimates**.

The players' usage estimates are numbers that take into account each player's average usage rate and minutes played. For example, if a player's average usage rate is 20% this season, 30% last season, and 27% in the last 5 years, and the player played 80 games both this season and the last, then

$$ \textup{player game usage coeff} = 0.6 \times \textup{this year's USG\%} + 0.2 \times \textup{last year's USG\%} + 0.2 \times \textup{5 year USG\%}. $$

There is also a minutes played usage coefficient, computed as follows: we take the player's average minutes coefficient computed earlier for both this season and last season (so if a player played at least 35 minutes in each game he played, both are $1$), compute a weight $\textup{minutes weight} =  \min(1, \textup{games played this season}  \times 0.1)$, and compute:

$$\textup{player game minutes coeff} = (\textup{minutes weight}) \times (\textup{avg minutes coeff this season}) + (1 - \textup{minutes weight}) \times (\textup{avg minutes coeff last season}).$$

Then we get each player's overall weight for the game by multiplying these:

$$\textup{player game weight} = \textup{player games usage coeff} \times \textup{player game minutes coeff}.$$

We take these game weights together into a vector $v_{\textup{weights, unnormalized}}$, normalize it:

$$v_{\textup{weights}} = \frac{v_{\textup{weights, unnormalized}}} {||v_{\textup{weights, unnormalized}}||_2},$$

where the $2$-norm was chosen because it worked best (the initial idea was the $1$-norm; I tuned this value too). Finally, we take the dot product of $v_{\textup{ratings}}$ and $v_{\textup{weights}}$:

$$\textup{team score} = \langle v_{\textup{ratings}}, v_{\textup{weights}} \rangle.$$

Usually, the best players' ratings are around 40, with quick fall-offs towards the 20's. Negative ratings are not uncommon! The prediction is that the team with the greater score wins. There should also be a home-court advantage factor - the one that worked best for the 2023-24 season is +3 for the home team.

In [5]:

import pandas as pd
import datetime
import numpy as np
import math
from IPython.display import clear_output
import unicodedata
from rapidfuzz import fuzz, process
from pathlib import Path


def split_up_totals(table):
    """
    Used to get the last row of a box score separatey and remove it
    """
    table_totals = table.iloc[-1]
    table = table.drop(index=table.index[-1])
    return table, table_totals


def get_cdf(value, dictionary):
    """
    Gets the value of the empirical CDF based on a histogram in the form of a dictionary. The dict. keys are values of a discrete RV,
    while the values of the dict. are the number of occurrences of that value of the RV. Used to get usg_coeff, gmsc_coeff and rtg_coeff.
    """
    tot = 0
    s = 0
    for key in dictionary:
        tot += dictionary[key]
        if key <= value:
            s += dictionary[key]
    return float(s)/float(tot)


def get_weights(row):
    """
    Used to calculate the rating of a player. 0 -> performance so far this season, 1 -> performance last season, 2 -> performance 2 seasons ago, 
    no number -> career (rather, 5-year) performance.
    I chose the weights a bit arbitrarily and then tried to tune the most important ones; no improvements made.
    """
    gp0 = row['GP0']
    gp1 = row['GP1']
    gp2 = row['GP2']
    gp = row['GP']
    weights = -1
    gp0c = gp1c = gp2c = gpc = 0
    if gp > 0:
        gpc = 1
    if gp0 > 0:
        gp0c = 1
    if gp1 > 0:
        gp1c = 1
    if gp2 > 0:
        gp2c = 1
    match [gp0c, gp1c, gp2c, gpc]:
        case [0,0,0,0]:
            weights = 0
        case [0,0,0,1]:
            weights = [0,0,0,1]
        case [0,0,1,1]:
            weights = [0,0,0.5,0.5]
        case [0,1,0,1]:
            weights = [0,0.6,0,0.4]
        case [0,1,1,1]:
            weights = [0,0.4,0.3,0.3]
        case [1,0,0,1]:
            weights = [0.9,0,0,0.1]
        case [1,0,1,1]:
            weights = [0.65,0,0.1,0.25]
        case [1,1,0,1]:
            weights = [0.5,0.3,0,0.2]
        case [1,1,1,1]:
            weights = [0.5,0.2,0.1,0.2]
    if weights == -1:
        print(str(gp0) + " " + str(gp1) + " " + str(gp2) + " " + str(gp))
        raise ValueError("There's an error")
    if weights != 0:
        weights = np.array(weights)
    return weights


def get_usage_weights(row):
    """
    Used to get the "usage factor" of a player, i.e. to compute how much his rating should contribute to his team's rating when predicting the outcome
    of a game. Based on this season's performance, last season's performance and 5-yr performance.
    The triple is later @'ed with average usage rates from this season, last season and the 5-year career average.
    """
    gp0 = row['GP0']
    gp1 = row['GP1']
    gp = row['GP']
    gp0c = gp1c = gpc = 0
    weights = -1
    if gp0 > 0:
        gp0c = 1
    if 20 <= gp1 < 50:
        gp1c = 1
    elif gp1 >= 50:
        gp1c = 2
    if gp > 0:
        gpc = 1
    match [gp0c, gp1c, gpc]:
        case [0,0,0]:
            weights = 0
        case [0,0,1]:
            weights = [0,0,1]
        case [0,1,1]:
            weights = [0,0.3,0.7]
        case [0,2,1]:
            weights = [0,0.7,0.3]
        case [1,0,1]:
            weights = [0.7,0,0.3]
        case [1,1,1]:
            weights = [0.65,0.1,0.25]
        case [1,2,1]:
            weights = [0.6,0.2,0.2]
    if weights == -1:
        raise ValueError("There's an error")
    if weights != 0:
        weights = np.array(weights)
    return weights

def add_weighted_usage_column(row):
    """
    computes a player's weighted usage by dot-producting the list from the get_usage_weights fctn with the player's average usage rates
    from this season, last season and 5 seasons ago. Used in predict_winner_and_process
    """
    dot_product = 0
    if isinstance(row['usage_wt_list'], np.ndarray):
        el_0 = el_1 = 0
        if row['GP0'] > 0:
            el_0 = row['sumusage0']/row['GP0']
        if row['GP1'] > 0:
            el_1 = row['sumusage1']/row['GP1']
        el_2 = row['career_avgusg']
        dot_product = row['usage_wt_list']@np.array([el_0,el_1,el_2])
    return dot_product

def get_overall_min_coeff(row):
    """
    gets the minutes coefficient (used to compute the score in process_box_score) - returns dot product of weight vector ([1,0] after 10 games this season)
    and the average minutes coefficients for this and last season
    """
    weight0 = min(1, row['GP0']*0.1)
    weight1 = 1 - weight0
    return weight0 * row['average_minutes_coeff_0'] + weight1 * row['average_minutes_coeff_1']

def get_overall_rating(row):
    """
    gets the dot product of the 4 ratings and the weights chosen in get_weights
    """
    if isinstance(row['weights'], np.ndarray):
        return row['weights']@np.array([row['rating0'], row['rating1'], row['rating2'], row['career_rating']])
    else: 
        return 0



#stop_year is the year we stop our training set - process_game function
def process_game(row, b, usg, gmsc, rtg, players_table, is_in_training, stop_year):
    """
    Important function that processes a row of the games dataframe. One row of 'games' contains basic info about a game. First we get the date and the
    teams that played, then we read the box scores for that game from the "Box_Scores" folder. We only use home_bs to get the team totals row,
    and we only need the team's total minutes played from that row (which is the same for both teams, hence we don't need the away box score).
    away_new and home_new are box score versions that only contain the info I need (minutes played, usage %, offRtg, defRtg, game score).
    Finally, in the end I call process_box_score twice - see below.
    """
    date = row['game_date']
    home_team = row['matchup_home'][:3]
    away_team = row['matchup_home'][-3:]
    year = date.year
    month = date.month
    day = date.day
    if month < 10:
        month = "0" + str(date.month)
    if day < 10:
        day = "0" + str(date.day)
    #Read the box scores. 
    home_bs = pd.read_csv(base_dir / "data" / "Box_Scores" / (str(year) + str(month) + str(day) + home_team + away_team + "_2.csv"))
    away_new = pd.read_csv(base_dir / "data" / "Box_Scores" / (str(year) + str(month) + str(day) + home_team + away_team + "_4.csv"))
    home_new = pd.read_csv(base_dir / "data" / "Box_Scores" / (str(year) + str(month) + str(day) + home_team + away_team + "_5.csv"))
    clear_output(wait=True)
    print(str(year) + str(month) + str(day) + home_team + away_team)
    #The last row of home_bs is the team's totals (total min_played, rebounds, assists, points, etc). We only need MP from it, basically to see
    #if there was overtime or not.
    home_bs, home_totals = split_up_totals(home_bs)
    total_team_mins = int(home_totals['MP'])
    #Process the box scores and modify the players table accordingly
    players_table = process_box_score(home_team, date, b, home_new, players_table, total_team_mins, usg, gmsc, rtg, is_in_training, stop_year)
    players_table = process_box_score(away_team, date, b, away_new, players_table, total_team_mins, usg, gmsc, rtg, is_in_training, stop_year)
    return players_table
    #home_new.apply(lambda row: change_rating(row, home_team, year, b, usg, gmsc, rtg, total_team_mins, stop_year, date, is_in_training), axis=1)
    #away_new.apply(lambda row: change_rating(row, away_team, year, b, usg, gmsc, rtg, total_team_mins, stop_year, date, is_in_training), axis=1)

def add_coeffs_and_score_to_box_score(box_score, team_mins, usg, gmsc, rtg, b):
    box_score[['minutes', 'seconds']] = box_score['MP'].str.split(':', expand=True)
    box_score['min_played'] = box_score['minutes'].astype(float) + (box_score['seconds'].astype(float)/60)    
    box_score['minutes_coeff'] = np.minimum(1, (6.85*box_score['min_played']/team_mins))
    box_score['usg_coeff'] = box_score['USG%'].astype(float).map(usg)
    box_score['gmsc_coeff'] = box_score['GmSc'].astype(float).map(gmsc)*20
    box_score['rtg_coeff'] = (box_score['ORtg'].astype(int) - box_score['DRtg'].astype(int)).map(rtg)*20
    box_score['score'] = box_score['usg_coeff']*box_score['minutes_coeff']*(b*box_score['gmsc_coeff'] + (1-b)*box_score['rtg_coeff'])*5    
    box_score['score'] = np.maximum(-100, box_score['score'])
    box_score['score'] = np.minimum(100, box_score['score'])
    return box_score

def modify_players_to_process_table(date, players_to_process, is_in_training, stop_year):
    year = date.year
    #'GP' = career games played, 'sum' = sum of all scores in the last 6 years, 'sumusage' = sum of all usage rates in the last 6 years
    # Suffix 0 = only this season, suffix 1 = last season, suffix 2 = 2 seasons ago.
    players_to_process['GP'] += 1
    players_to_process['sum'] += players_to_process['score']
    players_to_process['sumusage'] += players_to_process['USG%']
    if not(is_in_training):
        players_to_process['GP0'] += 1   
        players_to_process['sum0'] += players_to_process['score']   
        players_to_process['sumusage0'] += players_to_process['USG%']
        players_to_process['rating0'] =  players_to_process['sum0'].astype(float)/players_to_process['GP0'].astype(float)
        players_to_process['minutes_coeff_sum_0'] += players_to_process['minutes_coeff']
        players_to_process['average_minutes_coeff_0'] = players_to_process['minutes_coeff_sum_0'].astype(float)/players_to_process['GP0'].astype(float)
    else:
        k = stop_year - year
        #The season starts in October and ends in June (but in some exceptional cases it lasted until September).
        if date.month >= 10 and k <= 1:
            players_to_process['GP' + str(k+1)] += 1
            players_to_process['sum' + str(k+1)] += players_to_process['score']
            players_to_process['rating' + str(k+1)] = players_to_process['sum' + str(k+1)].astype(float)/players_to_process['GP' + str(k+1)].astype(float)
            if k == 0:
                players_to_process['sumusage1'] += players_to_process['USG%'].astype(float)
                players_to_process['minutes_coeff_sum_1'] += players_to_process['minutes_coeff']
                players_to_process['average_minutes_coeff_1'] = players_to_process['minutes_coeff_sum_1'].astype(float)/players_to_process['GP1'].astype(float)
        elif date.month <= 9 and k <= 0: 
            players_to_process['GP' + str(k+2)] += 1
            players_to_process['sum' + str(k+2)] += players_to_process['score']
            players_to_process['rating' + str(k+2)] = (players_to_process['sum' + str(k+2)].astype(float))/(players_to_process['GP' + str(k+2)].astype(float))
            if k == -1:
                players_to_process['sumusage1'] += players_to_process['USG%'].astype(float)
                players_to_process['minutes_coeff_sum_1'] += players_to_process['minutes_coeff']
                players_to_process['average_minutes_coeff_1'] = players_to_process['minutes_coeff_sum_1'].astype(float)/players_to_process['GP1'].astype(float)
    players_to_process['career_rating'] = players_to_process['sum'].astype(float)/players_to_process['GP'].astype(float)
    players_to_process['career_avgusg'] = players_to_process['sumusage'].astype(float)/players_to_process['GP'].astype(float)
    # Drop box_score columns from the DataFrame
    players_to_process = players_to_process.drop(columns=['GmSc', 'MP',
       'USG%', 'ORtg', 'DRtg', 'minutes', 'seconds', 'min_played',
       'minutes_coeff', 'usg_coeff', 'gmsc_coeff', 'rtg_coeff', 'score'])
    return players_to_process



def process_box_score(team, date, b, box_score, players_table, team_mins, usg, gmsc, rtg, is_in_training, stop_year):
    """
    This one's a bit annoying. First we find the players that played in the big players table and remove duplicates. Result: players_to_process DF
    Then we add some columns to box_score which contain the values we actually need to compute the rating ('minutes_coeff', 'usg_coeff', 'gmsc_coeff',
    'rtg_coeff', 'score').
    We drop a useless column 'Unnamed: 0' which is present in both box_score and players_to_process and creates confusion when merging.
    Then we merge the two dataframes along player names.
    Then we update the data (GP = games played, sum = sum of scores, GP0 = games played this season, etc.).
    If is_in_training, that means it's not the *current* season, so we'd update e.g. GP1,GP2,GP but not GP0.
    If not is_in_training, then it's the *current* season, so we update GP0, rating0 etc.
    Finally, we drop the box score columns from players_to_process, we drop the rows of players_to_process from players, and we add back the modified
    versions of the rows.
    """
    year = date.year
    month = date.month
    #Find players, remove duplicates
    players_to_process = players_table[players_table['Player'].isin(box_score['Starters'])]
    players_to_process = players_to_process[~(players_to_process.duplicated(subset='Player', keep=False) & ((year > players_to_process['To']) | (year < players_to_process['From'] - 1)))]
    players_to_process = players_to_process.drop_duplicates(subset='Player', keep='first')
    players_to_process['Team'] = team  
    #Add necessary columns to box_score that help compute the rating
    box_score = add_coeffs_and_score_to_box_score(box_score, team_mins, usg, gmsc, rtg, b)
    #Prepare for merging by renaming column, drop useless column 'Unnamed: 0', merge
    box_score.rename(columns={'Starters': 'Player'}, inplace=True)
    players_to_process = players_to_process.drop(columns=['Unnamed: 0'], errors='ignore')
    box_score = box_score.drop(columns=['Unnamed: 0'], errors='ignore')    
    players_to_process = pd.merge(players_to_process, box_score, on='Player', how='inner')
    #modify the values of the players_to_process_table by modifying the players' ratings and other auxiliary parameters
    players_to_process = modify_players_to_process_table(date, players_to_process, is_in_training, stop_year)
    # Initialize an empty list to hold the indices to drop
    indices_to_replace = []
    # Iterate through each row of player_rows_to_process
    for idx, row in players_to_process.iterrows():
        # Find the indices in players that match both 'Player' and 'From' values from the current row
        matching_indices = players_table[(players_table['Player'] == row['Player']) & (players_table['From'] == row['From'])].index
        # Append the matching indices to the list
        indices_to_replace.extend(matching_indices)
    # Drop the original rows from players
    players_table = players_table.drop(indices_to_replace)
    players_table = pd.concat([players_table, players_to_process], ignore_index=True)
    return players_table

#injury functions
def injury_updates(injury_table, date, players_table, injury_mask):
    """
    Injuries explained: I have an injury table with a list of all injuries that ever happened, together with the player's name and the dates.
    There are 2 rows in the table for each injury: when it happened, and when the player recovered. I edited the injuries table so that
    all dates (both injury & recovery) are on dates when there were games, because I don't want to think about dates which do not appear in the
    games table.
    Since there are multiple games on most dates, I only want to check for injuries on the *first* occurrence of each date. So if injury_mask[date] == False,
    that means it's the *first* game of that date and that I *want* to update the injury statuses of players. (there is a Boolean column 'injured' in
    the players table). Once the injuries for that day have been processed, I set injury_mask[date] to True, and for the remaining games on that date,
    the program won't check for injuries.
    """
    if injury_mask[date] == False:
        injuries_that_date = injury_table[injury_table['Date']==date]
        for idx, row in injuries_that_date.iterrows():
            players_table = process_injury_row(row, date, players_table)
        injury_mask[date] = True
    return players_table, injury_mask


def remove_duplicates_injuries(name_row_in_players, year):
    name_row_in_players_start = name_row_in_players
    name_row_in_players_temp = name_row_in_players
    if name_row_in_players.shape[0] > 1:
        players_removed = 0
        for idx, doppelganger in name_row_in_players.iterrows():
            if  (doppelganger['injured'] != player_returning) or (year > doppelganger['To']) or (year < doppelganger['From'] - 1) or (doppelganger['Team'] != team):
                name_row_in_players_temp = name_row_in_players_temp.drop(idx-players_removed)
                name_row_in_players_temp.reset_index(drop=True)
                players_removed += 1
        name_row_in_players = name_row_in_players_temp
    if isinstance(name_row_in_players, pd.DataFrame):
        try:
            name_row_in_players = name_row_in_players.iloc[0]
        except IndexError as e:
            print(name_row_in_players_start)
    return name_row_in_players


def process_injury_row(row, date, players_table):
    """
    In the injuries table, there are 4 notable columns: 'Date', 'Team', 'Acquired', 'Relinquished'.
    If the player in question *recovered* on that date, his name is in the 'Acquired' column.
    If he was injured on that date, his name is in the 'Relinquished' columns. One of the columns is always empty.
    I update the players table, in particular the player in question's 'injured' column.
    """
    clear_output(wait=True)
    year = date.year
    #In each row of the injury table, a player has either recovered ('Acquired') or gotten injured ('Relinquished')
    name = row['Acquired']
    team = row['Team']
    player_returning = True
    if pd.isna(name):
        name = row['Relinquished']
        player_returning = False
    print(name)
    print(date)
    print(team)
    #Get the right player in the players table
    name_row_in_players = players_table[players_table['Player']==name]
    name_row_in_players = name_row_in_players.reset_index(drop=True)
    #Remove duplicates
    name_row_in_players = remove_duplicates_injuries(name_row_in_players, year)
    #Create a Boolean mask to change the players table
    mask = (players_table[['Player', 'From']] == name_row_in_players[['Player', 'From']]).all(axis=1)
    #If the player is returning from a long injury, make sure he's not listed as retired
    if player_returning and date.month >= 10:
        players_table.loc[mask, 'To'] = year + 1
    if player_returning and date.month <= 9:
        players_table.loc[mask, 'To'] = year
    #Set the player's injury status
    players_table.loc[mask, 'injured'] = (not player_returning)
    return players_table

#transaction functions
def transaction_updates(transaction_table, date, players_table, transaction_mask):
    if transaction_mask[date] == False:
        transactions_that_date = transaction_table[transaction_table['Date']==date]
        for idx, row in transactions_that_date.iterrows():
            players_table = replace_team(row, players_table)
        transaction_mask[date] = True
    return players_table, transaction_mask

def fuzzy_match(name, choices):
    """
    Important for approximate matches - č/ć sometimes replaced by c, etc. Especially in the transactions and injuries tables.
    """
    match = process.extractOne(name, choices, scorer=fuzz.WRatio)
    return match[0]

def replace_team(row, players_table):
    """
    Go through transactions row by row and change the team of the corresponding row of the players table.
    """
    clear_output(wait=True)
    name = row['Player']
    new_team = row['New Team']
    date = row['Date']
    name_row_in_players = players_table[players_table['Player']==name]
    name_row_in_players = name_row_in_players.reset_index(drop=True)
    if not name_row_in_players.empty:
        #Remove duplicates
        name_row_in_players = remove_duplicates_transactions(name_row_in_players, date.year)
        #Create a Boolean mask to change the players table
        mask = (players_table[['Player', 'From']] == name_row_in_players[['Player', 'From']]).all(axis=1)
        players_table.loc[mask, 'Team'] = new_team
    return players_table


def remove_duplicates_transactions(name_row_in_players, year):
    name_row_in_players = name_row_in_players.reset_index(drop=True)
    name_row_in_players_temp = name_row_in_players.copy()
    if name_row_in_players.shape[0] > 1:
        print(name_row_in_players)
        players_removed = 0
        for idx, doppelganger in name_row_in_players.iterrows():
            if  (year > doppelganger['To']) or (year < doppelganger['From'] - 1):
                name_row_in_players_temp = name_row_in_players_temp.drop(idx-players_removed)
                name_row_in_players_temp = name_row_in_players_temp.reset_index(drop=True)
                players_removed += 1
        name_row_in_players = name_row_in_players_temp
    name_row_in_players = name_row_in_players.reset_index(drop=True)
    if isinstance(name_row_in_players, pd.DataFrame):
        if name_row_in_players.empty:
            raise ValueError("There's a problem - we removed all duplicates!")
        name_row_in_players = name_row_in_players.iloc[0]
    return name_row_in_players


def compute_ratings(home_players, away_players, home_weights):
    #We figure out how to assign weights of the previous seasons (both rating weights and usage weights) via their games played
    home_players['weights'] = home_players.apply(get_weights, axis=1)
    away_players['weights'] = away_players.apply(get_weights, axis=1)
    home_players['usage_wt_list'] = home_players.apply(get_usage_weights, axis=1)
    away_players['usage_wt_list'] = away_players.apply(get_usage_weights, axis=1)
    #We assign weights for the sum based on their usage rates
    home_players['usage_wt'] = home_players.apply(add_weighted_usage_column, axis=1)
    away_players['usage_wt'] = away_players.apply(add_weighted_usage_column, axis=1)
    home_players['minutes_coeff'] = home_players.apply(get_overall_min_coeff, axis=1)
    away_players['minutes_coeff'] = away_players.apply(get_overall_min_coeff, axis=1)
    home_players['multiplied_wt'] = home_players['usage_wt']*home_players['minutes_coeff']
    away_players['multiplied_wt'] = away_players['usage_wt']*away_players['minutes_coeff']
    home_wt_array = np.array(home_players['multiplied_wt'])
    away_wt_array = np.array(away_players['multiplied_wt'])
    home_wt_array = home_wt_array/np.linalg.norm(home_wt_array)
    away_wt_array = away_wt_array/np.linalg.norm(away_wt_array)
    #We compute the weighted average and predict the winner!
    home_players['weighted_rating'] = home_players.apply(get_overall_rating, axis=1)
    away_players['weighted_rating'] = away_players.apply(get_overall_rating, axis=1)
    home_ratings_array = np.array(home_players['weighted_rating'])
    away_ratings_array = np.array(away_players['weighted_rating'])
    home_rating = home_wt_array@home_ratings_array
    away_rating = away_wt_array@away_ratings_array
    home_weights.append(home_wt_array)
    return home_ratings_array, away_ratings_array, home_rating, away_rating


def check_for_team_name_change(date, players_table):
    if date == datetime.datetime(2012, 10, 30):
        players_table.loc[players_table['Team'] == 'NJN', 'Team'] = 'BRK'
        players_table.loc[players_table['Team'] == 'NOH', 'Team'] = 'NOP'
    if date == datetime.datetime(2014, 10, 28):
        players_table.loc[players_table['Team'] == 'CHA', 'Team'] = 'CHO'
    return players_table
    

def predict_winner_and_process(row, b, hca, usage_coeffs, gmsc_coeffs, rtg_coeffs, players_table, injury_mask, injuries, transaction_mask, transactions, is_in_training, stop_year, home_weights):
    """
    We get the appropriate weights for how much a player's score (itself weighted by his performance in the prev. seasons) should impact the team's score.
    These are based on a player's minutes played and usage rate.
    Then we compute the weighted average and predict the winner (the home team gets a 3-point bonus; this value was tuned)
    After predicting, we process the game and update the players table.
    """
    #First we get injury updates and see who's playing
    date = row['game_date']
    players_table, injury_mask = injury_updates(injuries, date, players_table, injury_mask)
    players_table, transaction_mask = transaction_updates(transactions, date, players_table, transaction_mask)
    players_table = check_for_team_name_change(date, players_table)
    home_team = row['matchup_home'][:3]
    away_team = row['matchup_home'][-3:]
    year = date.year
    month = date.month
    day = date.day
    #Then we identify the teams' players *who aren't injured* (or retired)
    home_players = players_table[(players_table['Team']==home_team) & (players_table['injured']==False) & (players_table['To']>=year)].copy()
    away_players = players_table[(players_table['Team']==away_team) & (players_table['injured']==False) & (players_table['To']>=year)].copy()
    #We compute their scores
    home_players_ratings, away_players_ratings, home_rating, away_rating = compute_ratings(home_players, away_players, home_weights)
    #We add the relevant columns to the games row
    home_ratings_sorted = np.sort(home_players_ratings)[::-1]
    away_ratings_sorted = np.sort(away_players_ratings)[::-1]
    row['rating_home'] = home_rating
    row['rating_away'] = away_rating
    for i in list(range(1,6)):
        row['rating_home_player' + str(i)] = home_ratings_sorted[i-1]
        row['rating_away_player' + str(i)] = away_ratings_sorted[i-1]
    #Finally, we process the game and get the new data before moving on to the next one...
    players_table = process_game(row, b, usage_coeffs, gmsc_coeffs, rtg_coeffs, players_table, is_in_training, stop_year)
    return row, players_table, injury_mask, transaction_mask

In [8]:
import pandas as pd
import datetime
import numpy as np
import math
from IPython.display import clear_output
import unicodedata
from rapidfuzz import fuzz, process
from pathlib import Path

# Define the base directory (relative to the current script)
base_dir = Path.cwd()

years = list(range(2008,2020))
for year in years:
    home_weights = []
    #Take in the start of training, end of training
    start_training_year = year
    end_training_year = year + 4
    
    #initialize data (games, players, injuries tables, total_games, correct_predictions)
    games = pd.read_csv(base_dir / "data" / "games_new_wl%.csv")
    games['game_date'] = pd.to_datetime(games['game_date'])
    games = games[games['game_date']>=datetime.datetime(start_training_year,10,1)]
    train_games = games[games['game_date'] <= datetime.datetime(end_training_year+1, 9, 30)]
    test_games = games[(games['game_date'] >= datetime.datetime(end_training_year+1, 10, 1)) & (games['game_date'] <= datetime.datetime(end_training_year+2,9,30))]
    first_test_game_date = test_games['game_date'].min()
    last_test_game_date = test_games['game_date'].max()
    players = pd.read_csv(base_dir / ("all_players.csv"))
    players['From'] = players['From'].astype(int)
    players['To'] = players['To'].fillna(0).astype(int)
    players = players.loc[(players['To'] >= start_training_year) | (players['From'] >= 2003)]
    injuries = pd.read_csv(base_dir / "data" / "all_injuries.csv")
    injuries['Date'] = pd.to_datetime(injuries['Date'])
    injuries = injuries[(injuries['Date'] >= datetime.datetime(start_training_year,10,1)) & (injuries['Date'] <= datetime.datetime(2024,12,16))]
    transactions = pd.read_csv(base_dir / ("transactions_teams_changed.csv"))
    transactions['Date'] = pd.to_datetime(transactions['Date'])
    transactions = transactions[(transactions['Date'] >= datetime.datetime(end_training_year+1,10,1)) & (transactions['Date'] <= datetime.datetime(end_training_year+2, 9, 30))]
    
    #add columns to games
    test_games['rating_home'] = 0
    test_games['rating_away'] = 0
    for i in list(range(1,6)):
        test_games['rating_home_player' + str(i)] = 0
        test_games['rating_away_player' + str(i)] = 0
    
    #add a bunch of columns to players
    players['GP'] = 0
    players['sum'] = 0
    players['GP0'] = 0
    players['sum0'] = 0
    players['GP1'] = 0
    players['sum1'] = 0
    players['GP2'] = 0
    players['sum2'] = 0
    players['injured'] = False
    players['sumusage'] = 0
    players['sumusage0'] = 0
    players['sumusage1'] = 0
    players['career_rating'] = 0
    players['career_avgusg'] = 0
    players['rating0'] = 0
    players['rating1'] = 0
    players['rating2'] = 0
    players['minutes_coeff_sum_1'] = 0
    players['minutes_coeff_sum_0'] = 0
    players['average_minutes_coeff_1'] = 0
    players['average_minutes_coeff_0'] = 0
    
    #get usage rate, game score, offrtg-defrtg histograms. I have them stored separately for each season - they are then summed over all training seasons.
    #There are 3 of them - usage rating, game score and (Offensive Rating minus Defensive Rating). They help determine how big a player's usage rate/gamescore/offdefrtg was,
    #compared to all other performances in the training set. That way, I don't have to judge what constitutes a good/bad performance, the data does it for me.
    end_year = end_training_year + 1
    usage_histos_list = [pd.read_csv(base_dir / "data" / "Histograms" / ("usg_rts_histogram_" + str(year) + "-" + str(year + 1) + ".csv")) for year in list(range(start_training_year,end_year))]
    gmsc_histos_list = [pd.read_csv(base_dir / "data" / "Histograms" / ("gmscs_histogram_" + str(year) + "-" + str(year + 1) + ".csv")) for year in list(range(start_training_year,end_year))]
    rtg_histos_list = [pd.read_csv(base_dir / "data" / "Histograms" / ("ortg-drtg_histogram_" + str(year) + "-" + str(year + 1) + ".csv")) for year in list(range(start_training_year,end_year))]
    usage_histos_list[0].columns = ['score', 'frequency']
    gmsc_histos_list[0].columns = ['score', 'frequency']
    rtg_histos_list[0].columns = ['score', 'frequency']
    usage_scores = usage_histos_list[0]['score']
    gmsc_scores = gmsc_histos_list[0]['score']
    rtg_scores = rtg_histos_list[0]['score']
    usage_histo = {score: 0 for score in usage_scores}
    gmsc_histo = {score: 0 for score in gmsc_scores}
    rtg_histo = {score: 0 for score in rtg_scores}
    for usg_histo_season, gmsc_histo_season, rtg_histo_season in zip(usage_histos_list, gmsc_histos_list, rtg_histos_list):
        usg_histo_season.columns = ['score', 'frequency']
        gmsc_histo_season.columns = ['score', 'frequency']
        rtg_histo_season.columns = ['score', 'frequency']
        for key in usage_histo:
            mask = (usg_histo_season['score']==key)
            usage_histo[key] += usg_histo_season.loc[mask, 'frequency']
        for key in gmsc_histo:
            mask = (gmsc_histo_season['score']==key)
            gmsc_histo[key] += gmsc_histo_season.loc[mask, 'frequency']
        for key in rtg_histo:
            mask = (rtg_histo_season['score']==key)
            rtg_histo[key] += rtg_histo_season.loc[mask, 'frequency']
    for key in usage_histo:
        usage_histo[key] = int(usage_histo[key].iloc[0])
    for key in gmsc_histo:
        gmsc_histo[key] = int(gmsc_histo[key].iloc[0])
    for key in rtg_histo:
        rtg_histo[key] = int(rtg_histo[key].iloc[0])
    
    
    #turn histograms into coefficient tables
    gmsc_coeffs = {score : 0 for score in gmsc_scores}
    for key in gmsc_coeffs:
        gmsc_coeffs[key] = 2*get_cdf(key, gmsc_histo)-1
    odrtg_coeffs = {diff: 0 for diff in rtg_scores}
    for key in odrtg_coeffs:
        odrtg_coeffs[key] = 2*get_cdf(key, rtg_histo)-1
    usgrt_coeffs = {rate : 0 for rate in usage_scores}
    for key in usgrt_coeffs:
        usgrt_coeffs[key] = get_cdf(key, usage_histo)
    
    
    #declare dates, injury mask
    all_dates = pd.date_range(first_test_game_date, last_test_game_date).tolist()
    injury_mask = {date: False for date in all_dates}
    transaction_mask = {date: False for date in all_dates}
    
    
    #predictions - run testing
    #The b_value controls how much we take into account game score (20%) and how much we take into account OffRtg-DefRtg (80%) - this was tuned
    b_value = 0.2
    #hca = home court advantage
    hca_value = 3
    #run training - the data has already been trained, and since I don't want to upload a 200MB file to github, I just do the testing here :)
    for idx, row in train_games.iterrows():
        players = process_game(row, b_value, usgrt_coeffs, gmsc_coeffs, odrtg_coeffs, players, True, end_training_year)
    train_games.to_csv(base_dir / ("trained_data_games_" + str(start_training_year) + "_" + str(end_training_year) + ".csv"))
    players.to_csv(base_dir / ("trained_data_players_" + str(start_training_year) + "_" + str(end_training_year) + ".csv"))
    #run testing
    for idx, row in test_games.iterrows():
        test_games.loc[idx], players, injury_mask, transaction_mask = predict_winner_and_process(row, b_value, hca_value, usgrt_coeffs, gmsc_coeffs, odrtg_coeffs, players, injury_mask, injuries, transaction_mask, transactions, False, end_training_year, home_weights)
    players.to_csv(base_dir / ("players_season_" + str(end_training_year+1) + "-" + str(end_training_year + 2) + ".csv"))
    test_games.to_csv(base_dir / ("season_" + str(end_training_year+1) + "-" + str(end_training_year + 2) + ".csv"))

20241216LACUTA


## 6. Web Scraping Transactions and Running Step 5

I found out in the meantime that basketball-reference has a list of all transactions in every NBA season! 

They can be found at https://www.basketball-reference.com/leagues/NBA_2024_transactions.html for the 2023-24 seasons. Change the year for different seasons.

Here I web scrape all transactions into a new DF: transactions_df, and the corresponding csv file is "all_transactions.csv". There are 3 columns: 'Player', 'Date' and 'New Team'. The 'New Team' value is 'n' for players who were waived or who retired.

I chose to enter the 3-team trades by hand!

Since a lot of the players who were waived or traded have never played an NBA game, I had to add all players who were ever drafted to my all_players table. Hence the lines of the form

    draft_tables = pd.read_html("https://www.basketball-reference.com/draft/NBA_" + str(year) + ".html")

... Who knows, they might play an NBA game one day.

I incorporated "live transactions" into the rating system above. Since the ratings are calculated game by game, and many transactions happen on days when there are no games, I changed the date of each transactions to the next date when there is a game. Of course, the transaction processing is done *before* the game processing. Also, since the players' names in the transactions table are sometimes not exact matches of those in the players table, I used rapidfuzz to find the closest match. Finally, since the teams in the rating system are represented by 3 letters, I modified them appropriately in the transactions table.

Once I finished processing the transactions, I incorporated them into the ratings system and went back to step 5.

I ran the ratings system in the following way: I wanted to add 12 columns to each row of games_new: 2 columns 'rating_home' and 'rating_away', and the ratings of the 5 best players of each team.

I ran 5 years of training (getting the 5-year ratings) for each year, and one year of computing ratings and adding the 12 columns, in a for loop (for year in list(range(2012,2025)): ...). In the first iteration, for example, the system trained from 2008-09 to 2012-13 and computed ratings for the 2013-14 season. 

The outputs of this process are csv files: "season_2013-2014.csv" up to "season_2024-2025.csv". In the last cell, I merge them together into one DataFrame: **seasons**. I export this into **all_seasons.csv**.

In [ ]:
import requests

years = list(range(2008,2026))

for year in years:
    # URL of the webpage
    url = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + "_transactions.html"
    
    # Send a GET request to fetch the HTML content
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Get the HTML content as a string
        html_content = response.text
        
        # Save the HTML content to a text file
        with open("transactions_" + str(year - 1) + ".txt", "w", encoding="utf-8") as file:
            file.write(html_content)
        print("HTML source code saved to 'webpage_source.txt'")
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")


In [ ]:
from datetime import datetime
import pandas as pd
from IPython.display import clear_output
from pathlib import Path

def find_next_occurrence(text, substrings, start_index):
    # Initialize the next index as None (not found yet)
    next_index = None
    next_substring = None
    # Iterate over the list of substrings
    for substring in substrings:
        index = text.find(substring, start_index)  # Find the substring starting from the given index
        if index != -1:  # Check if the substring was found
            if next_index is None or index < next_index:
                next_index = index
                next_substring = substring
    return next_substring, next_index  # Return the substring and its index

transactions_df = pd.DataFrame(columns=['Player', 'Date', 'New Team'])
print(transactions_df)

base_dir = Path.cwd()
years = list(range(2008,2026))
for year in years:
    index = 0
    with open("transactions_" + str(year - 1) + ".txt", "r", encoding="utf-8") as file:
        file_content = file.read()
        index = file_content.find("<h2>Transactions</h2>") + 22
        substrings = ["<li>", "<p>", "</ul>"]
        next_substring, index = find_next_occurrence(file_content, substrings, index)
        current_date = datetime(1,1,1)
        index_released = file_content.find("released")
        while next_substring != "</ul>": 
            if next_substring == "<li>":
                beginning_date_index = index + 10
                end_date_index = file_content.find("</span>", beginning_date_index)
                date_substring = file_content[beginning_date_index:end_date_index]
                if date_substring == "October ?, 2018":
                    date_substring = "October 1, 2018"
                if date_substring == "?":
                    date_substring = "October 1, 2024"
                current_date = datetime.strptime(date_substring, "%B %d, %Y")
                index = end_date_index
            if next_substring == "<p>":
                end_index = file_content.find("</p>",index+3)
                transaction = file_content[index+3:end_index]
                #index = end_index + 1
                verbs = ['released', 'signed', 'waived', 'traded', 'assigned', 'recalled', 'In a', 'appointed', 'hired', 'sold', 'claimed', 'retired', 'fired', 'resigns', 'suspended', 'announced', 'converted']
                verb, index_verb = find_next_occurrence(transaction, verbs, 0)
                index_verb += (index + 6)
                if verb == 'released' or verb == 'waived' or verb == 'assigned':
                    index_player = file_content.find(">", index_verb)
                    player_name = file_content[index_player+1:file_content.find("<", index_player)]
                    if player_name != "":
                        transactions_df.loc[len(transactions_df)] = [player_name, current_date, 'n']
                if verb == 'signed' or verb == 'recalled' or verb == 'claimed':
                    index_team = transaction.find(">", 0)
                    index_team_end = transaction.find("<", index_team)
                    team_name = transaction[index_team + 1: index_team_end]
                    index_gr = transaction.find(">", index_team_end)
                    index_gr2 = transaction.find(">", index_gr + 1)
                    index_less = transaction.find("<", index_gr2)
                    player_name = transaction[index_gr2 + 1: index_less]
                    if (verb != 'recalled') or ((verb == 'recalled') and not(('<' in player_name) or ('>' in player_name))):
                        transactions_df.loc[len(transactions_df)] = [player_name, current_date, team_name]
                if verb == 'traded':
                    index_tothe = transaction.find("to the", 0)
                    index_team1 = transaction.find(">", 0)
                    index_team1_end = transaction.find("<", index_team1)
                    team1_name = transaction[index_team1 + 1: index_team1_end]
                    index_team2 = transaction.find(">", index_tothe)
                    index_team2_end = transaction.find("<", index_team2)
                    team2_name = transaction[index_team2 + 1: index_team2_end]
                    index_players1 = index_team1_end
                    while index_players1 < index_tothe:
                        index_gr = transaction.find(">", index_players1)
                        index_gr2 = transaction.find(">", index_gr + 1)
                        index_less = transaction.find("<", index_gr2)
                        if index_gr2 < index_tothe:
                            player_name = transaction[index_gr2 + 1: index_less]
                            transactions_df.loc[len(transactions_df)] = [player_name, current_date, team2_name]
                        index_players1 = index_less
                    index_for = transaction.find(" for ", 0)
                    index_players2 = index_for
                    while (index_players2 < len(transaction)) and (index_players2 != -1):
                        index_gr = transaction.find(">", index_players2)
                        index_less = transaction.find("<", index_gr)
                        if (index_gr != -1) and (index_less != -1):
                            player_name = transaction[index_gr + 1: index_less]
                            if (player_name != " and ") and (not(',' in player_name)) and (not('draft' in player_name)):
                                transactions_df.loc[len(transactions_df)] = [player_name, current_date, team1_name]
                        index_players2 = transaction.find(">", index_less + 1)
                        if (index_gr == -1) or (index_less == -1):
                            index_players2 = -1
                if verb == 'sold':
                    index_gr = transaction.find(">", 0)
                    index_gr2 = transaction.find(">", index_gr+1)
                    index_gr3 = transaction.find(">", index_gr2+1)
                    index_less = transaction.find("<", index_gr3)
                    player_name = transaction[index_gr3 + 1: index_less]
                    index_gr4 = transaction.find(">", index_less)
                    index_gr5 = transaction.find(">", index_gr4+1)
                    index_less = transaction.find("<", index_gr5)
                    team_name = transaction[index_gr5 + 1: index_less]
                    transactions_df.loc[len(transactions_df)] = [player_name, current_date, team_name]
                if verb == 'retired' or verb == 'announced':
                    index_gr = transaction.find(">",0)
                    index_less = transaction.find("<",index_gr)
                    player_name = transaction[index_gr + 1: index_less]
                    transactions_df.loc[len(transactions_df)] = [player_name, current_date, 'n']
            next_substring, index = find_next_occurrence(file_content, substrings, index + 1)
transactions_df.to_csv(base_dir / "all_transactions.csv")

In [ ]:
from datetime import datetime
import pandas as pd
from IPython.display import clear_output
from pathlib import Path
import requests
import time

base_dir = Path.cwd()
years = list(range(1990,2025))
for year in years:
    time.sleep(2.1)
    # URL of the webpage
    draft_tables = pd.read_html("https://www.basketball-reference.com/draft/NBA_" + str(year) + ".html")
    print(draft_tables[0])
    draft_tables[0].to_csv(base_dir / ("draft_" + str(year) + ".csv"))

In [ ]:
import numpy as np

players = pd.read_csv(base_dir / "all_players.csv")
for year in years:
    draft = pd.read_csv(base_dir / ("draft_" + str(year) + ".csv"))
    #draft.columns = draft.iloc[0]
    #draft = draft.iloc[1:]
    #draft = draft[draft['Player'] != "Round 2"]
    #draft = draft[draft['Player'] != "Player"]
    #draft = draft[draft['Player'] != ""]
    for idx, row in draft.iterrows():
        if row['Player'] not in players['Player'].values:
            players.loc[len(players)] = [np.nan, row['Player'], year, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, row['Pk'], 0, row['Tm']]
    players.to_csv(base_dir / "all_players_even_the_ones_who've_never_played.csv")

In [ ]:
import datetime

all_dates = pd.date_range(start="2007-01-01", end="2024-12-31").to_list()
date_masks = {date: False for date in all_dates}
games = pd.read_csv(base_dir / "all_games.csv")
games['game_date'] = pd.to_datetime(games['game_date'])
for key in date_masks:
    if key in games['game_date'].values:
        date_masks[key] = True

In [ ]:
transactions = pd.read_csv(base_dir / "all_transactions.csv")
transactions['Date'] = pd.to_datetime(transactions['Date'])
transactions.reset_index(drop=True, inplace=True)
for idx, row in transactions.iterrows():
    print(row)
    if not date_masks[row['Date']]:
        date = row['Date']
        while not date_masks[date]:
            date += datetime.timedelta(days=1)
        transactions.loc[idx, 'Date'] = date
transactions

In [ ]:
transactions.to_csv(base_dir / "all_transactions_date_edited.csv")

In [ ]:
from rapidfuzz import fuzz, process

players = pd.read_csv(base_dir / "all_players.csv")
transactions = pd.read_csv(base_dir / "all_transactions_date_edited.csv")
indices_to_drop = []
for idx, transaction in transactions.iterrows():
    print(idx)
    transaction_player = transaction['Player']

    # Find the closest match for the player in the players DataFrame
    match, score, _ = process.extractOne(transaction_player, players['Player'], scorer=fuzz.ratio)

    if score > 65:
        # Update the 'Player' column in transactions
        transactions.at[idx, 'Player'] = match
    else: indices_to_drop.append(idx)

transactions = transactions.drop(indices_to_drop)

transactions.to_csv(base_dir / "all_transactions_names_edited.csv")

In [14]:
transactions['New Team'].unique()

array(['n', 'Indiana Pacers', 'Miami Heat', 'Boston Celtics',
       'Toronto Raptors', 'Phoenix Suns', 'Portland Trail Blazers',
       'Seattle SuperSonics', 'Orlando Magic', 'San Antonio Spurs',
       'Houston Rockets', 'Utah Jazz', 'Sacramento Kings',
       'Denver Nuggets', 'Memphis Grizzlies', 'New Jersey Nets',
       'Milwaukee Bucks', 'Chicago Bulls', 'Los Angeles Lakers',
       'New Orleans Hornets', 'Dallas Mavericks',
       'Minnesota Timberwolves', 'Golden State Warriors',
       'Los Angeles Clippers', 'Detroit Pistons', 'Washington Wizards',
       'Philadelphia 76ers', 'Cleveland Cavaliers', 'Atlanta Hawks',
       'Charlotte Bobcats', 'New York Knicks', 'Oklahoma City Thunder',
       'Brooklyn Nets', 'New Orleans Pelicans', 'Charlotte Hornets', nan],
      dtype=object)

In [ ]:
dict_teams = {"Indiana Pacers": "IND", "Miami Heat": "MIA", "Boston Celtics": "BOS", "Toronto Raptors": "TOR", "Phoenix Suns": "PHO",
       "Portland Trail Blazers": "POR", "Seattle SuperSonics": "OKC", "Orlando Magic": "ORL", "San Antonio Spurs": "SAS", "Houston Rockets": "HOU",
       "Utah Jazz": "UTA", "Sacramento Kings": "SAC", "Denver Nuggets": "DEN", "Memphis Grizzlies": "MEM", "New Jersey Nets": "NJN", "Milwaukee Bucks": "MIL",
        "Chicago Bulls": "CHI", "Los Angeles Lakers": "LAL", "New Orleans Hornets": "NOH", "Dallas Mavericks": "DAL", "Minnesota Timberwolves": "MIN", 
                'Golden State Warriors': "GSW", 'Los Angeles Clippers': "LAC", 'Detroit Pistons': "DET", 'Washington Wizards': "WAS", 'Philadelphia 76ers': "PHI",
                'Cleveland Cavaliers': "CLE", 'Atlanta Hawks': "ATL", 'Charlotte Bobcats': "CHA", "New York Knicks": "NYK", "Oklahoma City Thunder": "OKC",
                "Brooklyn Nets": "BRK", "New Orleans Pelicans": "NOP", "Charlotte Hornets": "CHO"}

transactions = pd.read_csv(base_dir / "all_transactions_names_edited.csv")
for idx, row in transactions.iterrows():
    if (row['New Team'] != 'n') and (row['New Team'] != np.nan):
        print(str(idx) + " " + row['Player'])
        transactions.loc[idx, 'New Team'] = dict_teams[row['New Team']]
transactions.to_csv(base_dir / "transactions_teams_changed.csv")

In [8]:
from pathlib import Path

base_dir = Path.cwd()
years = list(range(2014,2025))
seasons = pd.read_csv(base_dir / "season_2013-2014.csv")
for year in years:
    new_season = pd.read_csv(base_dir / ("season_" + str(year) + "-" + str(year+1) + ".csv"))
    seasons = pd.concat([seasons, new_season])
seasons

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,game_id,game_date,team_name_home,team_name_away,matchup_home,wl_home,record_home_wins,...,rating_home_player1,rating_away_player1,rating_home_player2,rating_away_player2,rating_home_player3,rating_away_player3,rating_home_player4,rating_away_player4,rating_home_player5,rating_away_player5
0,56375,56375,56375,21300003,2013-10-29,Los Angeles Lakers,Los Angeles Clippers,LAL vs. LAC,W,0,...,17.111626,32.202332,10.780722,22.505542,2.765120,6.884314,2.324031,6.382166,1.885043,6.129456
1,56376,56376,56376,21300001,2013-10-29,Indiana Pacers,Orlando Magic,IND vs. ORL,W,0,...,15.778079,6.266414,12.472956,4.322534,11.424487,3.901378,10.933116,1.463819,6.154944,0.457323
2,56377,56377,56377,21300002,2013-10-29,Miami Heat,Chicago Bulls,MIA vs. CHI,W,0,...,46.642413,23.312382,27.254811,14.034707,17.901815,12.147442,8.753337,11.150026,5.179129,6.078947
3,56378,56378,56378,21300010,2013-10-30,Minnesota Timberwolves,Orlando Magic,MIN vs. ORL,W,0,...,20.016182,16.419562,14.661371,7.546233,13.104385,4.565434,6.309203,1.463819,5.774210,-0.928863
4,56379,56379,56379,21300013,2013-10-30,San Antonio Spurs,Memphis Grizzlies,SAS vs. MEM,W,0,...,22.746048,16.920225,22.001119,14.965448,12.988484,12.853586,9.954069,4.892916,8.711481,4.638953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,70895,70895,70895,22400384,2024-12-16,Detroit Pistons,Miami Heat,DET vs. MIA,W,10,...,7.988674,15.274545,5.177711,12.486273,2.797891,2.060638,2.262335,1.802428,2.089380,0.591959
385,70896,70896,70896,22400385,2024-12-16,Brooklyn Nets,Cleveland Cavaliers,BRK vs. CLE,L,10,...,7.938384,19.949751,3.345870,17.854959,2.331521,6.758387,1.856738,5.857342,1.848113,2.842600
386,70897,70897,70897,22400386,2024-12-16,Toronto Raptors,Chicago Bulls,TOR vs. CHI,L,7,...,11.967490,17.039529,3.822437,8.351248,2.382569,3.996091,2.345124,3.194908,2.278176,2.927639
387,70898,70898,70898,22400387,2024-12-16,Sacramento Kings,Denver Nuggets,SAC vs. DEN,L,13,...,25.864927,41.113787,20.087284,11.392381,15.664944,11.370816,6.393516,5.160928,4.494256,1.225719


In [10]:
seasons.to_csv(base_dir / "all_seasons.csv")

## 7. Team Offensive and Defensive Ratings as Columns

The purpose of the remaining cells was to add 4 more columns to games_new (actually, by now, the main table became seasons): 'ortg_home', 'ortg_away', 'ortg_home_last_season', 'ortg_away_last_season'. As explained in step 5, a team's offensive (resp. defensive) rating is the number of points they score (resp. allow) per 100 possessions. I wanted to add the last_season versions too because these stats don't mean much in the beginning of the season.

First I had to web scrape the **pace** (number of possessions / 2) of each game. This is usually a bit less than 100 and can be found in each box score page on basketball reference (but not in the box score itself). I added the pace, points scored and points allowed to each team's season total while iterating through the rows of the seasons DF. Then I divided them by the number of games played to compute the average. Before doing this, though, I added the average values up to that point to the current row. At the end of each season, I saved each team's OffRtg and DefRtg in a csv file ("odrtgs_" + str(year) + "-" + str(year+1) + ".csv"). 

When I finished iterating through the seasons DF, I had two new columns: 'ortg_home' and 'ortg_away'. Finally, I added the previous season's OffRtgs and DefRtgs to each row, using the csv files above.

The end result was "games_almost_ready_for_training.csv".

In [6]:
import requests
import time
from IPython.display import clear_output
import datetime
import pandas as pd

games = pd.read_csv(base_dir / "all_games.csv")
games['game_date'] = pd.to_datetime(games['game_date'])
seasonsfrom201213 = games[games['game_date'] >= datetime.datetime(2012,10,1)]
for idx, row in seasonfroms201213.iterrows():
    time.sleep(2.1)
    clear_output(wait=True)
    home_team = row['matchup_home'][:3]
    away_team = row['matchup_home'][-3:]
    date = row['game_date']
    year = date.year
    month = date.month
    day = date.day
    if month < 10:
        month = "0" + str(month)
    if day < 10:
        day = "0" + str(day)
    # URL of the webpage
    url = "https://www.basketball-reference.com/boxscores/" + str(year) + str(month) + str(day) + "0" + home_team + ".html"
    
    # Send a GET request to fetch the HTML content
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Get the HTML content as a string
        html_content = response.text
        
        # Save the HTML content to a text file
        with open(str(year) + str(month) + str(day) + home_team + away_team + "HTML.txt", "w", encoding="utf-8") as file:
            file.write(html_content)
        print("HTML source code saved to " + str(year) + str(month) + str(day) + home_team + away_team + "HTML.txt")
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        print(url)

HTML source code saved to 20130620MIASASHTML.txt


In [9]:
from IPython.display import clear_output

games = pd.read_csv(base_dir / "all_games.csv")
games['game_date'] = pd.to_datetime(games['game_date'])
seasons_to_process = games[games['game_date'] >= datetime.datetime(2012,10,1)].copy()
seasons_to_process['pace'] = 0.0
for idx, row in seasons_to_process.iterrows():
    clear_output(wait=True)
    home_team = row['matchup_home'][:3]
    away_team = row['matchup_home'][-3:]
    date = row['game_date']
    print(date)
    year = date.year
    month = date.month
    day = date.day
    if month < 10:
        month = "0" + str(month)
    if day < 10:
        day = "0" + str(day)
    with open("HTML for pace and rtg/" + str(year) + str(month) + str(day) + home_team + away_team + "HTML.txt", "r", encoding="utf-8") as file:
        content = file.read()
        index = content.find('data-stat="pace"')
        index = content.find('data-stat="pace"', index + 1)
        index = content.find('>', index + 1)
        index_gr = content.find('<', index + 1)
        pace = float(content[index+1 : index_gr])
        print(pace)
        seasons_to_process.loc[idx, 'pace'] = pace
seasons_to_process

2024-12-16 00:00:00
103.7


,Unnamed: 0,index,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,...,pf_away,pts_away,plus_minus_away,video_available_away,season_type,record_home_wins,record_home_losses,record_away_wins,record_away_losses,pace
55061,55061,55883,22012,NaN,NaN,Cleveland Cavaliers,21200000,2012-10-30,CLE vs. WAS,W,...,NaN,84,NaN,NaN,Regular Season,1,0,0,1,88.9
55062,55062,55884,22012,NaN,NaN,Miami Heat,21200001,2012-10-30,MIA vs. BOS,W,...,NaN,107,NaN,NaN,Regular Season,1,0,0,1,95.0
55063,55063,55885,22012,NaN,NaN,Los Angeles Lakers,21200002,2012-10-30,LAL vs. DAL,L,...,NaN,99,NaN,NaN,Regular Season,0,1,1,0,91.6
55064,55064,55886,22012,NaN,NaN,Philadelphia 76ers,21200003,2012-10-31,PHI vs. DEN,W,...,NaN,75,NaN,NaN,Regular Season,1,0,0,1,94.4
55065,55065,55887,22012,NaN,NaN,Toronto Raptors,21200004,2012-10-31,TOR vs. IND,L,...,NaN,90,NaN,NaN,Regular Season,0,1,1,0,91.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70895,70895,72556,22024,NaN,NaN,Detroit Pistons,22400384,2024-12-16,DET vs. MIA,W,...,NaN,124,NaN,NaN,Regular Season,11,16,13,11,101.9
70896,70896,72557,22024,NaN,NaN,Brooklyn Nets,22400385,2024-12-16,BRK vs. CLE,L,...,NaN,130,NaN,NaN,Regular Season,10,16,23,4,98.9
70897,70897,72558,22024,NaN,NaN,Toronto Raptors,22400386,2024-12-16,TOR vs. CHI,L,...,NaN,122,NaN,NaN,Regular Season,7,20,12,15,108.2
70898,70898,72559,22024,NaN,NaN,Sacramento Kings,22400387,2024-12-16,SAC vs. DEN,L,...,NaN,130,NaN,NaN,Regular Season,13,14,14,10,108.8


In [11]:
years = list(range(2012,2025))
teams_2012_13 = ['CHA', 'TOR', 'BOS', 'BRK', 'PHI', 'NYK', 'CLE', 'CHI', 'IND', 'MIL', 'DET', 'ATL', 'WAS', 'MIA', 'ORL',
                 'POR', 'OKC', 'UTA', 'DEN', 'MIN', 'GSW', 'LAC', 'PHO', 'SAC', 'LAL', 'HOU', 'MEM', 'SAS', 'DAL', 'NOH']
teams_2013_14 = ['CHA', 'TOR', 'BOS', 'BRK', 'PHI', 'NYK', 'CLE', 'CHI', 'IND', 'MIL', 'DET', 'ATL', 'WAS', 'MIA', 'ORL',
                 'POR', 'OKC', 'UTA', 'DEN', 'MIN', 'GSW', 'LAC', 'PHO', 'SAC', 'LAL', 'HOU', 'MEM', 'SAS', 'DAL', 'NOP']
teams_now = ['CHO', 'TOR', 'BOS', 'BRK', 'PHI', 'NYK', 'CLE', 'CHI', 'IND', 'MIL', 'DET', 'ATL', 'WAS', 'MIA', 'ORL',
            'POR', 'OKC', 'UTA', 'DEN', 'MIN', 'GSW', 'LAC', 'PHO', 'SAC', 'LAL', 'HOU', 'MEM', 'SAS', 'DAL', 'NOP']
for year in years:
    season = seasons_to_process[(seasons_to_process['game_date'] >= datetime.datetime(year,10,1)) & (seasons_to_process['game_date'] <= datetime.datetime(year+1,9,30))].copy()
    season['ortg_home'] = 0.0
    season['drtg_home'] = 0.0
    season['ortg_away'] = 0.0
    season['drtg_away'] = 0.0
    poss_scored_allowed = {team : [0.0,0,0] for team in teams_now}
    if year == 2012:
        poss_scored_allowed = {team : [0.0,0,0] for team in teams_2012_13}
    if year == 2013:
        poss_scored_allowed = {team : [0.0,0,0] for team in teams_2013_14}
    for idx, row in season.iterrows():
        home_team = row['matchup_home'][:3]
        away_team = row['matchup_home'][-3:]
        season.loc[idx, 'ortg_home'] = 0
        season.loc[idx, 'drtg_home'] = 0
        if poss_scored_allowed[home_team][0] != 0:
            season.loc[idx, 'ortg_home'] = (float(poss_scored_allowed[home_team][1])/poss_scored_allowed[home_team][0])*100
            season.loc[idx, 'drtg_home'] = (float(poss_scored_allowed[home_team][2])/poss_scored_allowed[home_team][0])*100
        season.loc[idx, 'ortg_away'] = 0
        season.loc[idx, 'drtg_away'] = 0
        if poss_scored_allowed[away_team][0] != 0:
            season.loc[idx, 'ortg_away'] = (float(poss_scored_allowed[away_team][1])/poss_scored_allowed[away_team][0])*100
            season.loc[idx, 'drtg_away'] = (float(poss_scored_allowed[away_team][2])/poss_scored_allowed[away_team][0])*100
        poss_scored_allowed[home_team][0] += row['pace']
        poss_scored_allowed[away_team][0] += row['pace']
        poss_scored_allowed[home_team][1] += row['pts_home']
        poss_scored_allowed[away_team][1] += row['pts_away']
        poss_scored_allowed[home_team][2] += row['pts_away']
        poss_scored_allowed[away_team][2] += row['pts_home']
    odrtgs_season = {team: [(float(poss_scored_allowed[team][1])/poss_scored_allowed[team][0])*100, (float(poss_scored_allowed[team][2])/poss_scored_allowed[team][0])*100] for team in list(poss_scored_allowed.keys())}
    odrtgs_df = pd.DataFrame(columns=['team', 'ortg', 'drtg'])
    for team in odrtgs_season:
        odrtgs_df.loc[len(odrtgs_df)] = [team, odrtgs_season[team][0], odrtgs_season[team][1]]
    odrtgs_df.to_csv(base_dir / ("odrtgs_" + str(year) + "-" + str(year+1) + ".csv"))
    season.to_csv(base_dir / ("season_with_odrtgs_" + str(year) + "-" + str(year+1) + ".csv"))

In [57]:
games_with_odrtg = pd.read_csv(base_dir / "season_with_odrtgs_2013-2014.csv")
for year in list(range(2014,2025)):
    this_season = pd.read_csv(base_dir / ("season_with_odrtgs_" + str(year) + "-" + str(year+1) + ".csv"))
    games_with_odrtg = pd.concat([games_with_odrtg, this_season])
games_with_odrtg.to_csv(base_dir / "games_with_odrtg.csv")

In [63]:
games_odrtg = games_with_odrtg[['ortg_home', 'drtg_home', 'ortg_away', 'drtg_away']]
#games_with_odrtg.reset_index(inplace=True)
#seasons.reset_index(inplace=True)
seasons = pd.concat([seasons, games_odrtg], axis=1)

In [64]:
seasons

,level_0,index,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,game_id,game_date,team_name_home,team_name_away,matchup_home,...,rating_home_player3,rating_away_player3,rating_home_player4,rating_away_player4,rating_home_player5,rating_away_player5,ortg_home,drtg_home,ortg_away,drtg_away
0,0,0,56375,56375,56375,21300003,2013-10-29,Los Angeles Lakers,Los Angeles Clippers,LAL vs. LAC,...,2.765120,6.884314,2.324031,6.382166,1.885043,6.129456,0.000000,0.000000,0.000000,0.000000
1,1,1,56376,56376,56376,21300001,2013-10-29,Indiana Pacers,Orlando Magic,IND vs. ORL,...,11.424487,3.901378,10.933116,1.463819,6.154944,0.457323,0.000000,0.000000,0.000000,0.000000
2,2,2,56377,56377,56377,21300002,2013-10-29,Miami Heat,Chicago Bulls,MIA vs. CHI,...,17.901815,12.147442,8.753337,11.150026,5.179129,6.078947,0.000000,0.000000,0.000000,0.000000
3,3,3,56378,56378,56378,21300010,2013-10-30,Minnesota Timberwolves,Orlando Magic,MIN vs. ORL,...,13.104385,4.565434,6.309203,1.463819,5.774210,-0.928863,0.000000,0.000000,90.155440,100.518135
4,4,4,56379,56379,56379,21300013,2013-10-30,San Antonio Spurs,Memphis Grizzlies,SAS vs. MEM,...,12.988484,12.853586,9.954069,4.892916,8.711481,4.638953,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14520,14520,384,70895,70895,70895,22400384,2024-12-16,Detroit Pistons,Miami Heat,DET vs. MIA,...,2.797891,2.060638,2.262335,1.802428,2.089380,0.591959,112.138547,115.547371,115.959596,112.592593
14521,14521,385,70896,70896,70896,22400385,2024-12-16,Brooklyn Nets,Cleveland Cavaliers,BRK vs. CLE,...,2.331521,6.758387,1.856738,5.857342,1.848113,2.842600,113.935542,118.309743,120.902740,111.000230
14522,14522,386,70897,70897,70897,22400386,2024-12-16,Toronto Raptors,Chicago Bulls,TOR vs. CHI,...,2.382569,3.996091,2.345124,3.194908,2.278176,2.927639,112.724035,117.833779,113.793866,116.853436
14523,14523,387,70898,70898,70898,22400387,2024-12-16,Sacramento Kings,Denver Nuggets,SAC vs. DEN,...,15.664944,11.370816,6.393516,5.160928,4.494256,1.225719,117.934104,114.732979,117.411866,114.660361


In [65]:
seasons['home_ortg_last_season'] = 0.0
seasons['home_drtg_last_season'] = 0.0
seasons['away_ortg_last_season'] = 0.0
seasons['away_drtg_last_season'] = 0.0
years = list(range(2013,2025))
new_df = 0
first_time = True
for year in years:
    odrtgs_last_season = pd.read_csv(base_dir / ("odrtgs_" + str(year-1) + "-" + str(year) + ".csv"))
    odrtgs_last_season = odrtgs_last_season.set_index('team')
    seasons['game_date'] = pd.to_datetime(seasons['game_date'])
    this_season = seasons[(seasons['game_date'] >= datetime.datetime(year,10,1)) & (seasons['game_date'] <= datetime.datetime(year+1,9,30))]
    for idx, row in this_season.iterrows():
        home_team = row['matchup_home'][:3]
        away_team = row['matchup_home'][-3:]
        if year == 2013:
            if home_team == 'NOP':
                home_team = 'NOH'
            if away_team == 'NOP':
                away_team = 'NOH'
        if year == 2014:
            if home_team == 'CHO':
                home_team = 'CHA'
            if away_team == 'CHO':
                away_team = 'CHA'
        this_season.loc[idx, 'home_ortg_last_season'] = odrtgs_last_season.loc[home_team, 'ortg']
        this_season.loc[idx, 'home_drtg_last_season'] = odrtgs_last_season.loc[home_team, 'drtg']
        this_season.loc[idx, 'away_ortg_last_season'] = odrtgs_last_season.loc[away_team, 'ortg']
        this_season.loc[idx, 'away_drtg_last_season'] = odrtgs_last_season.loc[away_team, 'drtg']
    if first_time:
        new_df = this_season
        first_time = False
    else:
        new_df = pd.concat([new_df, this_season])
new_df

,level_0,index,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,game_id,game_date,team_name_home,team_name_away,matchup_home,...,rating_home_player5,rating_away_player5,ortg_home,drtg_home,ortg_away,drtg_away,home_ortg_last_season,home_drtg_last_season,away_ortg_last_season,away_drtg_last_season
0,0,0,56375,56375,56375,21300003,2013-10-29,Los Angeles Lakers,Los Angeles Clippers,LAL vs. LAC,...,1.885043,6.129456,0.000000,0.000000,0.000000,0.000000,107.555523,107.308892,110.690362,104.491602
1,1,1,56376,56376,56376,21300001,2013-10-29,Indiana Pacers,Orlando Magic,IND vs. ORL,...,6.154944,0.457323,0.000000,0.000000,0.000000,0.000000,105.034238,101.365066,102.131827,109.714301
2,2,2,56377,56377,56377,21300002,2013-10-29,Miami Heat,Chicago Bulls,MIA vs. CHI,...,5.179129,6.078947,0.000000,0.000000,0.000000,0.000000,113.022477,104.622588,104.454612,105.099688
3,3,3,56378,56378,56378,21300010,2013-10-30,Minnesota Timberwolves,Orlando Magic,MIN vs. ORL,...,5.774210,-0.928863,0.000000,0.000000,90.155440,100.518135,103.134360,105.682833,102.131827,109.714301
4,4,4,56379,56379,56379,21300013,2013-10-30,San Antonio Spurs,Memphis Grizzlies,SAS vs. MEM,...,8.711481,4.638953,0.000000,0.000000,0.000000,0.000000,109.914713,102.916827,105.857799,101.788588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14520,14520,384,70895,70895,70895,22400384,2024-12-16,Detroit Pistons,Miami Heat,DET vs. MIA,...,2.089380,0.591959,112.138547,115.547371,115.959596,112.592593,110.148046,119.280187,113.804461,112.796409
14521,14521,385,70896,70896,70896,22400385,2024-12-16,Brooklyn Nets,Cleveland Cavaliers,BRK vs. CLE,...,1.848113,2.842600,113.935542,118.309743,120.902740,111.000230,113.915287,116.898483,114.703355,113.260338
14522,14522,386,70897,70897,70897,22400386,2024-12-16,Toronto Raptors,Chicago Bulls,TOR vs. CHI,...,2.278176,2.927639,112.724035,117.833779,113.793866,116.853436,113.042945,119.521472,116.540378,118.073153
14523,14523,387,70898,70898,70898,22400387,2024-12-16,Sacramento Kings,Denver Nuggets,SAC vs. DEN,...,4.494256,1.225719,117.934104,114.732979,117.411866,114.660361,117.922423,115.992037,117.647059,113.159582


In [66]:
new_df.to_csv(base_dir / "games_almost_ready_for_training.csv")